# Numerical modelling of synaptic transmission in 3D

In [1]:
%matplotlib notebook 
import numpy as np
import matplotlib.pyplot as plt
from getCylinder2 import GetCylinder

from matplotlib.animation import FuncAnimation
from IPython.display import HTML, clear_output

In [2]:
%%html 
<style>
.output_wrapper button.btn.btn-default, .output_wrapper .ui-dialog-titlebar {
    display: none;
} </style>

In [3]:
#Constants, for now set everything equal to one

## 3D
* Build mass matrix and load vector
* Need new mesh generator, new matrix and load vector builder, otherwise the same
* Solve system of ODE's
* More complicated animation

In [ ]:
def animate_diffusion(nArray, fps, step, p, steps):
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    
    #Method to change the contour plot
    def animate(i):
        ax.clear()
        ax.scatter3D(p[:,0], p[:,1], p[:,2], c = nArray[i*step])
        ax.set_title(f"diffusion after {i*step} steps, i = {i}");

    ani = FuncAnimation(fig, animate, frames = steps-1, repeat = False)
    #return HTML(ani.to_jshtml())
    return ani

def animate_func(p, nArray, steps = 100, duration = 5):
    step = len(nArray)//steps
    plt.rcParams["figure.figsize"] = [5.00, 5.00]
    plt.rcParams["figure.autolayout"] = True
    fps = steps / duration
    animations = animate_diffusion(nArray, fps,step, p, steps)
    
    return animations

In [ ]:
%%capture
ani = animate_func(p, nArray, 20)

In [ ]:
display(HTML(ani.to_jshtml()))

### Trying diffusion only with Dirichlet Boundary conditions

### New attempt to get something working: Finite differences using quadratic base plate

You specify Number of nodes in x and y direction N, number of nodes in z direction n and height of z-axis.
The first and last (N+1)^2 nodes are on boundary. On each layer the (N+1) first and last are on boundary. Also for each N-1 row at each layer, the first and last are on boundary.

In [ ]:
def GenerateUniformMesh(N, n, h=0.1):
    #Get nodal values
    p = NodalPoints(N, n, h)
    
    # Get boundary nodes
    bottom_bd, top_bd, East_bd, West_bd, South_bd, North_bd, Interior = Boundary2(p, N, n)
    return p, bottom_bd, top_bd, East_bd, West_bd, South_bd, North_bd, Interior

def NodalPoints(N,n,h):
    #N number of nodes in x and y direction
    p = np.zeros([])
    for i in range(n+1):
        p_ = np.zeros(((N+1)**2, 3))
        for j in range(N+1):
            p_[j*(N+1):(j+1)*(N+1),0] = np.linspace(-1,1,N+1)
            p_[j*(N+1):(j+1)*(N+1),1] = np.ones(N+1)*-1+2*j*1/N
            p_[j*(N+1):(j+1)*(N+1),2] = np.ones(N+1)*i*h/n
            
        if i == 0:
            p = p_
        else:
            p = np.concatenate((p, p_), axis = 0)
    return p

def Boundary(p, N, n):
    bottom_bd = np.arange((N+1)**2)
    top_bd = bottom_bd + n*(N+1)**2
    East_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,0] == 1 and p[i,1] != -1 and p[i,1] != 1])
    West_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,0] == -1 and p[i,1] != -1 and p[i,1] != 1])
    South_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,1] == -1 and p[i,0] != -1 and p[i,0] != 1])
    North_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,1] == 1 and p[i,0] != -1 and p[i,0] != 1])
    Interior = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,1] != 1 and p[i,1] != -1 and p[i,0] != -1 and p[i,0] != 1])
    return bottom_bd, top_bd, East_bd, West_bd, South_bd, North_bd, Interior

def Boundary2(p, N, n):
    bottom_bd = np.arange((N+1)**2)
    bottom_bd = np.array([i for i in range((N+1)**2) if p[i,0] != 1 and p[i,0] != -1 and p[i,1] != -1 and p[i,1] != 1])
    top_bd = bottom_bd + n*(N+1)**2
    East_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,0] == 1 and p[i,1] != -1 and p[i,1] != 1])
    West_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,0] == -1 and p[i,1] != -1 and p[i,1] != 1])
    South_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,1] == -1 and p[i,0] != -1 and p[i,0] != 1])
    North_bd = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,1] == 1 and p[i,0] != -1 and p[i,0] != 1])
    Interior = np.array([i for i in range((N+1)**2, n*(N+1)**2) if p[i,1] != 1 and p[i,1] != -1 and p[i,0] != -1 and p[i,0] != 1])
    return bottom_bd, top_bd, East_bd, West_bd, South_bd, North_bd, Interior

In [ ]:
N = 10
n = 5

p, bottom, top, east, west, south, north, Interior = GenerateUniformMesh(N, n)
args = (bottom ,top, east, west, south, north)
boundary = np.concatenate(args)

In [ ]:
middle = int(1/2 * n*(N+1)**2 + ((N+1)**2)//2)
print(p[middle])
print(p[middle-(N+1)])
#fig = plt.figure()
plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(p[boundary,0], p[boundary,1], p[boundary,2])

#fig.close()

In [ ]:
def FiniteDifferenceApproxDirichlet(N, n, h = 0.1, maxiter = 1000):
    # Generate mesh
    p, bottom, top, east, west, south, north, interior = GenerateUniformMesh(N, n)
    args = (bottom ,top, east, west, south, north)
    boundary = np.concatenate(args)
    middle = int(1/2 * n*(N+1)**2 + ((N+1)**2)//2)
    # Initial conditions
    # first try: middle point has value
    n0 = np.zeros(len(p)) # Full matrix
    n0[middle] = 1
    n_interior0 = n0[interior]
    
    # Finite differences for interior points
    # For now homogenous Dirichlet
    dx, dy = 2/N, 2/N
    dz = h/n
    A = GenerateA(dx, dy, dz, N, n)
    I = np.identity(len(A))
    nArray = [n0]
    dt = 0.5/maxiter
    for i in range(maxiter):
        n_interior = nArray[i][interior]        
        n_int_new = np.linalg.solve((I - 1/2 * dt * A), (I + 1/2 * dt * A)@n_interior)
        n_new = np.zeros(len(p))
        n_new[interior] = n_int_new
        nArray.append(n_new)
    # Homogenous Neumann BC part
    '''
    p[bottom] = p[bottom+(N+1)**2] # Derivative in z-direction is 0
    p[top] = p[top-(N+1)**2] # Derivative in z-direction is 0
    p[east] = p[east-1] # Derivative in x-direction is 0
    p[west] = p[west+1] # Derivative in x-direction is 0
    p[south] = p[south + N+1] # Derivative in z-direction is 0
    p[north] = p[north - N-1] # Derivative in z-direction is 0
    ''';
    return p, nArray, boundary

def GenerateA(dx, dy, dz, N, n):
    alpha = -2*(1/(dx**2) + 1/(dy**2) + 1/(dx**2))
    A = np.zeros(((N-1)**2*(n-1), (N-1)**2*(n-1)))
    np.fill_diagonal(A, np.ones((N-1)**2*(n-1))*alpha)

    np.fill_diagonal(A[1:], np.ones((N-1)**2*(n-1)-1)*1/(dx**2))
    np.fill_diagonal(A[:,1:], np.ones((N-1)**2*(n-1)-1)*1/(dx**2))

    np.fill_diagonal(A[N-1:], np.ones((N-1)**2*(n-1)-N+1)*1/(dy**2))
    np.fill_diagonal(A[:,N-1:], np.ones((N-1)**2*(n-1)-N+1)*1/(dy**2))

    np.fill_diagonal(A[(N-1)**2:], np.ones((N-1)**2*(n-1)-(N-1)**2)*1/(dy**2))
    np.fill_diagonal(A[:,(N-1)**2:], np.ones((N-1)**2*(n-1)-(N-1)**2)*1/(dy**2))
    return A

In [ ]:
p, nArray, boundary = FiniteDifferenceApproxDirichlet(10, 10, 0.1, 1000)

In [ ]:
for n in nArray:
    if not (all(node == 0 for node in n[boundary])):
        print("Error")
        break

In [ ]:
%%capture
ani = animate_func(p, nArray, 50)

In [ ]:
display(HTML(ani.to_jshtml()))

In [ ]:
def findBoundaryIndexes(N_, n_):
    int_bottom = np.arange(N_**2 )
    int_top = np.arange(N_**2*(n_-1), N_**2*(n_))
    int_south_first = np.arange(N_)
    int_south = int_south_first
    for i in range(1,n_):
        int_south_ = int_south_first + i*N_**2
        int_south = np.concatenate((int_south, int_south_))
        
    int_north = int_south + N_**2 - N_
    int_west_first = np.arange(N_)*N_
    int_west = int_west_first
    for i in range(1, n_):
        int_west_ = int_west_first + i*N_**2
        int_west = np.concatenate((int_west, int_west_))
    int_east = int_west + N_-1
    return int_bottom, int_south, int_north, int_west, int_east, int_top

In [ ]:
def FiniteDifferenceApproxNeumann(N, n, h = 0.1, maxiter = 1000):
    # Generate mesh
    p, bottom, top, east, west, south, north, interior = GenerateUniformMesh(N, n)
    args = (bottom ,top, east, west, south, north)
    boundary = np.concatenate(args)
    middle = int(1/2 * n*(N+1)**2)
    # Initial conditions
    # first try: middle point has value
    n0 = np.zeros(len(p)) # Full matrix
    n0[middle] = 1
    n_interior0 = n0[interior]
    
    # Finite differences for interior points
    dx, dy = 2/N, 2/N
    dz = h/n
    A = GenerateA(dx, dy, dz, N, n)
    I = np.identity(len(A))
    
    # Need to find which nodes lie near boundary
    int_bottom, int_south, int_north, int_west, int_east, int_top = findBoundaryIndexes(N-1, n-1)
    
    nArray = [n0]
    dt = 1/maxiter
    for i in range(maxiter):
        n_interior = nArray[i][interior]
        # Create F that adds boundary terms
        # This is not quite right, int_bottom is not boundary of full matrix
        F = np.zeros(len(n_interior))
        
        F[int_bottom] += 1/(dz**2) * nArray[i][bottom]
        F[int_top] += 1/(dz**2) * nArray[i][top]
        F[int_south] += 1/(dy**2) * nArray[i][south]
        F[int_north] += 1/(dy**2) * nArray[i][north]
        F[int_west] += 1/(dx**2) * nArray[i][west]
        F[int_east] += 1/(dx**2) * nArray[i][east]
        
        g = np.zeros(len(n_interior))
        g[int_bottom] += 1/(dz**2)
        g[int_top] += 1/(dz**2)
        g[int_south] += 1/(dy**2)
        g[int_north] += 1/(dy**2)
        g[int_west] += 1/(dx**2)
        g[int_east] += 1/(dx**2)
        
        G = np.zeros(A.shape)
        np.fill_diagonal(G, g)
        #n_int_new = np.linalg.solve((I - 1/2 * dt * (A+G)), (I + 1/2 * dt * A)@n_interior+ dt*F)
        #n_int_new = np.linalg.solve((I - 1/2 * dt * A), (I + 1/2 * dt * A)@n_interior)# + dt*F)
        #n_int_new = (I + dt * A)@n_interior #+ dt*F
        n_new = np.zeros(len(p))
        n_new[interior] = n_int_new
        
        
        # Homogenous Neumann part, i.e derivative in direction of normal = 0
        n_new[bottom] = n_new[bottom + (N+1)**2]
        n_new[top] = n_new[top - (N+1)**2]
        n_new[south] = n_new[south + (N+1)]
        n_new[north] = n_new[north - (N+1)]
        n_new[west] = n_new[west + 1]
        n_new[east] = n_new[east - 1]
        
        
        nArray.append(n_new)
        
    return p, nArray, boundary

In [ ]:
N = 10
n = 5
p, nArray, boundary = FiniteDifferenceApproxNeumann(N, n, h = 0.1, maxiter = 10000)

In [ ]:
%%capture
ani = animate_func(p, nArray, 20)

In [ ]:
display(HTML(ani.to_jshtml()))

In [ ]:
for n_ in nArray:
    print(np.sum(n_))